In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

In [ ]:
base <- "193.175.238.88:8080/api/organizations/"
call1 <- paste(base,"all/", sep="")
get_organizations_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
# In the table 'get_organizations_df' you can see all the extracted organizations with respected twitter, facebook and wikipedia ids.

tables_rjs(get_organizations_df[get_organizations_df$category == 'media',], jupyter = TRUE)

In [ ]:
# Please choose a subcategory from the table above and input the `media` and `aggregation` variables below:
media <- "Newspaper"
aggregation <- "month" # aggregation can be by day ('day'), week ('week') or month('month')

media_df <- get_organizations_df[grepl(media, get_organizations_df$subcategory, fixed=TRUE) & !is.null(get_organizations_df$fb_ids) ,]

posts <- data.frame()
comments <- data.frame()
# make sure that accounts are not repeated, e.g. Die Zeit (Online) and Die Zeit (Allgemeine) could
# share an account if facebook but not in twitter
answer = c()
for (fb_id in media_df$fb_ids){
    for (fb in fb_id){
       
     answer <- append(answer, fb)
    }
        
}
fb_id<-answer[!duplicated(answer)]
# query the Social Media Monitoring API

for (fb in fb_id) {
    call_posts <- paste(base,"facebook/posts_by/organizations/user_id/", toString(fb), "/?from_date=2017-09-01&to_date=2018-12-31&aggregate_by=", aggregation, sep="")
    call_comments <- paste(base,"facebook/comments_by/organizations/user_id/", toString(fb), "/?from_date=2017-09-01&to_date=2018-12-31&aggregate_by=", aggregation, sep="")
    if (length(fromJSON(content(GET(call_posts), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
        posts_unique <- as.data.frame(fromJSON(content(GET(call_posts), "text", encoding="UTF-8"), flatten = TRUE))
        posts_unique$fb_user_id = fb
        posts <- rbind(posts, posts_unique)
        }
    if (length(fromJSON(content(GET(call_comments), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
        comments_unique <- as.data.frame(fromJSON(content(GET(call_comments), "text", encoding="UTF-8"), flatten = TRUE))
        comments_unique$fb_user_id = fb
        comments <- rbind(comments, comments_unique)
        }
}


posts <- summarise(group_by(posts, labels), posts = sum(values), shares = sum(shares), replies = sum(replies), reactions = sum(reactions), likes = sum(likes), response_type = aggregation)
comments <- summarise(group_by(comments, labels), comments = sum(values), replies = sum(replies), likes = sum(likes), response_type = aggregation)
merged <- merge(posts, comments, by='labels')

In [ ]:
#plotting facebook posts
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = posts, color="Posts", group=response_type.x)) +
  geom_line(aes(y = replies.x, color="Replies", group=response_type.x)) +
  geom_line(aes(y = shares, color="Shares", group=response_type.x)) +
  geom_line(aes(y = reactions, color="Reactions", group=response_type.x)) +
  geom_line(aes(y = likes.x, color="Likes", group=response_type.x)) +
  labs(title = "Facebook (Post Activity)", x = "", y = "Posts") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), plot.title = element_text(size=10)) +
  scale_x_date(date_breaks = "1 months")+ scale_y_continuous(labels = scales::number_format(big.mark = ""))


In [ ]:
#plotting facebook comments
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = comments, color="Comments", group=response_type.y)) +
  geom_line(aes(y = replies.y, color="Replies", group=response_type.y)) +
  geom_line(aes(y = likes.y, color="Likes", group=response_type.x)) +
  labs(title = "Facebook (Comments Activity)", x = "", y = "Comments") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), plot.title = element_text(size=10)) +
  scale_x_date(date_breaks = "1 months")+ scale_y_continuous(labels = scales::number_format(big.mark = ""))
